In [9]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity/')
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity/python_src/')

import numpy as np
import numpy.random as rand
import numpy.linalg as la
import scipy as sp
import scipy.sparse as sparse
import scipy.optimize as spo
import gzip
import cPickle as pickle
import itertools as it
import portalocker

import mech_network_solver as mns
import network as nw
import tuning_algs as talgs


import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib as mpl
import seaborn as sns

sns.set(color_codes=True)
sns.set_context('poster', font_scale=1.75)
sns.set_palette("hls", 8)
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})
mpl.rcParams['mathtext.fontset'] = 'cm'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
NN = 64
eta = 1e-1
NTS = 128
DIM = 2
Lp = -1.0
irec = 0


nw_label = "network_periodic_jammed/network_N{:05d}_Lp{:.2f}/network_irec{:04d}".format(NN, Lp, irec)

with gzip.open("/data1/home/rocks/data/{:}.pklz".format(nw_label), 'rb') as pkl_file:

    nw_data = pickle.load(pkl_file)
    net = nw_data['network']    

NF = 1

edgei = net.edgei
edgej = net.edgej


inodesi = [[] for t in range(NF)]
inodesj = [[] for t in range(NF)]

onodesi = [[] for t in range(NF)]
onodesj = [[] for t in range(NF)]

edge = range(net.NE)

rand.seed(2)

rand.shuffle(edge)

b = edge.pop()

inodesi[0].append(edgei[b])
inodesj[0].append(edgej[b])

for i in range(NTS):
    b = edge.pop()

    onodesi[0].append(edgei[b])
    onodesj[0].append(edgej[b])


print "inodes", inodesi, inodesj

print "onodes", onodesi, onodesj

isvec = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(inodesi[t], inodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        isvec[t].extend(bvec) 

istrain = [[] for t in range(NF)]
istrain[0].append(1.0)


osvec = [[] for t in range(NF)]
ostrain = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(onodesi[t], onodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        osvec[t].extend(bvec) 

        r = rand.randint(2)
        ostrain[t].append((2*r-1) * eta)

pert = []
meas = []
for t in range(NF):
    pert.append(talgs.Perturb())
    pert[t].setInputStrain(len(inodesi[t]), inodesi[t], inodesj[t], istrain[t], isvec[t])

    meas.append(talgs.Measure())
    meas[t].setOutputStrain(len(onodesi[t]), onodesi[t], onodesj[t], osvec[t])

obj_func = mns.CyAugIneqRatioChangeObjFunc(np.concatenate(ostrain))    


cypert = []
for p in pert:
    cypert.append(p.getCyPert())

cymeas = []
for m in meas:
    cymeas.append(m.getCyMeas())

K_init = np.ones(net.NE, float) / net.eq_length

solver = mns.CyLinSolver(net.getCyNetwork(), len(cypert), cypert, cymeas)
solver.setIntStrengths(K_init)

x_init = obj_func.initialize(solver)

# print x_init

NVars = len(x_init)

inodes [[28]] [[39]]
onodes [[22, 19, 35, 15, 20, 4, 11, 17, 20, 28, 37, 13, 47, 2, 30, 26, 60, 1, 7, 4, 60, 1, 7, 55, 6, 12, 18, 18, 9, 33, 27, 29, 17, 24, 7, 55, 8, 28, 47, 7, 39, 26, 19, 4, 8, 41, 21, 31, 55, 7, 23, 11, 21, 14, 14, 58, 9, 11, 56, 19, 48, 30, 35, 6, 25, 3, 18, 26, 24, 9, 23, 23, 41, 33, 45, 13, 56, 34, 36, 53, 20, 34, 57, 22, 6, 35, 61, 59, 4, 5, 16, 12, 6, 33, 38, 47, 21, 17, 46, 52, 35, 38, 51, 46, 22, 3, 13, 45, 5, 60, 44, 14, 0, 52, 12, 28, 57, 62, 4, 24, 0, 50, 30, 27, 6, 16, 15, 20]] [[42, 56, 63, 32, 57, 26, 37, 39, 50, 40, 32, 45, 38, 40, 38, 30, 53, 30, 37, 46, 56, 38, 20, 46, 32, 34, 41, 48, 2, 41, 54, 51, 34, 48, 50, 41, 50, 31, 25, 32, 40, 38, 62, 29, 59, 45, 2, 40, 44, 8, 53, 42, 25, 33, 18, 1, 56, 63, 53, 24, 62, 36, 37, 20, 48, 47, 45, 52, 62, 62, 36, 61, 43, 63, 48, 12, 62, 39, 53, 49, 32, 43, 12, 55, 19, 60, 54, 50, 44, 49, 40, 17, 57, 55, 31, 1, 48, 27, 0, 0, 58, 52, 59, 43, 44, 14, 41, 57, 10, 36, 46, 63, 28, 31, 27, 34, 27, 2, 51, 45, 43, 54, 51, 

In [16]:
def func(x):
    return obj_func.res(x, solver)

def jac(x):
    (res, rows, cols, vals) = obj_func.resGrad(x, solver)
        
    return sparse.coo_matrix((vals, (rows, cols)), shape=(len(res), NVars))
    
    

In [17]:
x_low = np.full(NVars, -np.inf)
x_low[0:net.NE] = 0.0

x_up = np.full(NVars, np.inf)
x_up[0:net.NE] = K_init

tol = tol = np.sqrt(np.finfo(float).eps)

res = spo.least_squares(func, x_init, jac=jac, 
                                    bounds=(x_low, x_up),
                                   method='trf', ftol=0.0, xtol=tol, gtol=0.0, max_nfev=1000000000, 
                                   verbose=2, tr_solver='lsmr', tr_options={'show':False, 'damp':0.0}, x_scale='jac')

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         6.4000e+01                                    3.39e+01    
       1              2         2.4545e+01      3.95e+01       5.14e-01       1.43e+01    
       2              3         1.5873e+01      8.67e+00       9.48e-01       1.09e+01    
       3              4         1.5246e+01      6.27e-01       8.82e-02       9.97e+00    
       4              5         1.5018e+01      2.28e-01       2.56e-02       9.07e+00    
       5              6         1.4513e+01      5.04e-01       8.25e-02       8.49e+00    
       6              7         1.4250e+01      2.64e-01       4.39e-02       7.44e+00    
       7              8         1.3373e+01      8.76e-01       1.90e-01       5.24e+00    
       8              9         1.3021e+01      3.52e-01       2.92e-01       5.17e+00    
       9             10         1.2737e+01      2.84e-01       6.30e-02       5.97e+00    

KeyboardInterrupt: 